# Spotted Python Week 8 Tutorial
In this script, we aim to practice the following skills:
- Fitting a Gaussian
- Determining redshift of a galaxy

In [ ]:
# import libraries
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy.io import ascii
from astropy.table import Table, join
from scipy.odr import *
from scipy.signal import find_peaks
import eazy

## Gaussian Fitting
In pervious weeks, we looked at linear fitting. However gaussian fitting is equally common within physics.

### **Example 1:**
Generate simulated spectral line and fit an gaussian

In [ ]:
# define a general gaussian function
def gaussian(parameters, x):
    c, A, mu, sigma = parameters
    return c + A * np.exp(-(x - mu) ** 2 / (2 * sigma ** 2))

In [ ]:
# generate simulated data
x_sim = np.linspace(1000, 1300, 100) # x values
y_sim = gaussian([10,23,1150,10],x_sim)
y_noise = np.random.normal(10,2,y_sim.shape) # add gaussian noise
y_total = y_sim + y_noise # final y_values

# plot simulated data to check shape
plt.plot(x_sim, y_total, color='royalblue')

In [ ]:
# Fitting a gaussian to the simulated data
model = Model(gaussian) # making the defined function into a model
# takes the real data and fits it to the model with an initial guess for the parameters
odr = ODR(RealData(x_sim, y_total),
                 model, beta0=[100, 20, 1150, 15])
output = odr.run()
output.pprint()

In [ ]:
# plot the simulated data with the fitted gaussian
plt.plot(x_sim, y_total, color = 'royalblue', label='simulated data') # orginal data
plt.plot(x_sim, gaussian(output.beta, x_sim), color='deeppink', label ='fitted gaussian')
plt.legend(loc='best') # show legend

### **Excercise 1:**
To practice this fitting, you can use the below space to simulate your own data.

## Spectroscopic Redshift
Recall that redshift can be determined using the following formulae: $$z=\frac{\Delta \lambda}{\lambda_{rest}}=\frac{\lambda_{observed}-\lambda_{rest}}{\lambda_{rest}}\Rightarrow z=\frac{\lambda_{observed}}{\lambda_{rest}}-1,$$ where $\lambda_{rest}$ is the rest frame wavelength, and $\lambda_{observed}$ is the oberserved wavelength in the spectra.

### **Example 2:**
Open NGC0000.fits, extract flux, plot the spectra, and identify any emission/absorption lines. Attempt to fit the line using a gaussian or alternative methods, and determine the redshift of the example spectra

Notes:
- List of spectral lines available [here](http://astronomy.nmsu.edu/drewski/tableofemissionlines.html#:~:text=The%20ultraviolet%20and%20optical%20spectra,nature%20of%20the%20ionizing%20source.)
- $H_{\alpha}$ (6562.819 A) is almost always the highest in flux, so let's identify that first
- To the left of $H_{\alpha}$ is $[NII]6548$ (6548.050 A)
- To the right of $H_{\alpha}$ is $[NII]6583$ (6583.460 A)

In [ ]:
# Open fits
data = fits.open('NGC0000.fits')
data.info()

In [ ]:
# Let's have a look at the headers
data['data'].header

In [ ]:
# To get the wavelength, we need to use CRPIX3, CRVAL3, CD3_3, and NAXIS3
crpix3 = data['data'].header['crpix3']
crval3 = data['data'].header['crval3']
cdelt3 = data['data'].header['cd3_3']
naxis3 = data['data'].header['naxis3']

wavelength = crval3 + (np.arange(naxis3) + 1 - crpix3) * cdelt3
print(f"wavelength range: [{wavelength[0]} to {wavelength[-1]} Angstroms]")

cube = data['data'].data
print(f"flux cube shape: {cube.shape}")

# Let's just look at the central pixel
flux = cube[:, 15, 15]

# Plot the spectra
plt.figure(figsize=(15,5))
plt.plot(wavelength, flux, color='darkviolet')
plt.xlabel('Wavelength, $\lambda$ $\\AA$') # x-axis label
plt.ylabel('Flux, $F$ $[10^{-20} \\AA cm^{-2} erg s^{-1}]$') # y-axis label

In [ ]:
# let's zoom into the wavelength range of interest
wavelength_short = wavelength[1450:1600]
flux_short = flux[1450:1600]
plt.plot(wavelength_short, flux_short, color='darkviolet')
plt.xlabel('Wavelength, $\lambda$ $\\AA$') # x-axis label
plt.ylabel('Flux, $F$ $[10^{-20} \\AA cm^{-2} erg s^{-1}]$') # y-axis label

In [ ]:
# define the rest frame wavelengths
h_alpha = 6562.819
nii_1 = 6548.050
nii_2 = 6583.460

# Identify the wavelengths of the spectral lines
peaks, _ = find_peaks(flux_short, height=1700)
print(f"number of peaks: {len(peaks)}")

# define the observed wavelengths
h_alpha_obs = wavelength_short[peaks[1]] # h_alpha
nii_1_obs = wavelength_short[peaks[0]] # nii 6548.050
nii_2_obs = wavelength_short[peaks[2]] # nii 6583.460

# Calculate redshift using individual lines
z_1 = h_alpha_obs/h_alpha -1
z_2 = nii_1_obs/nii_1 -1
z_3 = nii_2_obs/nii_2 -1
z_avg = np.mean((z_1,z_2,z_3))
print(f'Spectroscopic redshift is: z= {round(z_avg, 4)}') # prints to 4 decimal places

### **Exercise 2:**
Repeat example 2 with another spectra from the cube, you should get a redshift that is similar (since it is the same galaxy)

## SED Fitting


### **Example 3:**
AGEL211627-594702 is a strong gravitational lens as part of the AGEL survey sample (see figure below). Using source detection software, the flux of the sources across multiple bands (g, r, i, z, y) have been extracted and given in the file *AGEL211627-594702_flux.csv*. Below, we will go through an example of how photometric redshifts can be derived using SED fitting.

In [ ]:
# Dictionary of parameters to be used in EAZY
params = {}
params['CATALOG_FILE'] = 'AGEL211627-594702_flux.csv'
params['MAIN_OUTPUT_FILE'] = 'output/AGEL211627-594702_output.eazypy'

# Galactic extinction
params['MW_EBV'] = 0.0103 #degree of interstellar reddening E(B-V)=(B-V)-(B-V)_0

# Redshift steps
params['Z_STEP'] = 0.01
params['Z_MIN'] = 0.01
params['Z_MAX'] = 2

# Prior
params['PRIOR_ABZP'] = 22.5 # conversion factor from linear flux to magnitudes when flux is in nanomaggies
params['PRIOR_FILTER'] = 28 # K-band
params['PRIOR_FILE'] = 'templates/prior_K_TAO.dat'
params['TEMPLATES_FILE'] = 'templates/fsps_full/tweak_fsps_QSF_12_v3.param'
params['FIX_ZSPEC'] = False
params['IGM_SCALE_TAU'] = 1.0
translate_file = 'AGEL_translate.csv'

In [ ]:
# Run the photoZ class
self = eazy.photoz.PhotoZ(param_file=None, translate_file=translate_file, zeropoint_file=None,
                          params=params, load_prior=True, load_products=False)

# Fit the full catalog
sample = np.isfinite(self.ZSPEC)
self.fit_catalog(self.idx[sample], n_proc=8)
zout, hdu = self.standard_output(simple=False,
                                 rf_pad_width=0.5, rf_max_err=2,
                                 prior=True, beta_prior=True,
                                 absmag_filters=[],
                                 extra_rf_filters=[])

In [ ]:
# SED Fits
for s in range(0,len(zout['id'])):
    self.OBJID[s]=s
    ix = np.where(self.OBJID == s)[0][0]
    fig, data = self.show_fit(s, xlim=[0.4, 1.2], # wavelength ranges from 0.398 to 1.065 microns
                        show_components=True, add_label=True,
                        template_color='royalblue', logpz=True, zr=[0,params['Z_MAX']])